# 0. Setup

```bash
uv sync
uv run dreadnode login
```


## 1. Initialize Components

Set up Neo4j database, BBOT tool, and Strikes tracking


In [1]:
import dreadnode as dn
import rigging as rg
import litellm
from loguru import logger

from bbot_agent.tools import BbotTool, finish_task, update_todo, highlight_for_review
from bbot_agent.prompts import operator

# Configure Strikes
dn.configure(
    # api_key="your_api_key_here",  # or `uv run dreadnode login`
    project="agent-workshop",
    console=False,
)

# Create a BBOT tool to use Neo4j and containers
bbot_tool = BbotTool()

print("✓ Components initialized")

Dreadnode logging to https://platform.dreadnode.io (profile: demo)

14:23:30.163 | BBOT working out of /Users/nick/code/aisec-workshop/.bbot


✓ Components initialized


## 2. Start Services

Launch Neo4j container and connect BBOT tool


In [ ]:
# Verify BBOT + Neo4j work

async with bbot_tool:
    version = await bbot_tool.get_bbot_version()
    print(f"✓ BBOT {version} ready")

14:23:32.534 | Starting Neo4j container ...
14:23:32.550 | Image 'neo4j:latest' already exists locally
14:23:33.678 | Started container 'magical_hypatia' (affa7a147f72)
14:23:33.678 | Port mappings: {7687: 55011, 7474: 55010}
14:23:33.679 | Waiting for 'magical_hypatia' (affa7a147f72) to be healthy ...
14:23:38.406 | Neo4j container started 'magical_hypatia'
14:23:38.408 |  |- Dashboard: http://localhost:55010
14:23:38.409 |  |- Bolt URI:  bolt://localhost:55011
14:23:38.424 | Successfully connected to Neo4j at bolt://localhost:55011


✓ BBOT v2.5.0 ready
✓ Neo4j ready at bolt://localhost:55011


14:23:49.173 | Successfully stopped container 'magical_hypatia' (affa7a147f72)


## 3. Test Tool Methods

Verify individual tool capabilities before agent execution


In [ ]:
# Test basic scan functionality

async with bbot_tool:
    scan_result = await bbot_tool.run_scan(
        targets=["dreadnode.io"],
        modules=["httpx"],  # Quick web discovery
    )

In [ ]:
# Test Neo4j queries

async with bbot_tool:
    scans = await bbot_tool.get_scans()
    print(f"\n✓ Found {len(scans)} scans in database")

    print(scans)

## 4. Configure Agent Prompt

Use the operator prompt template with Jinja2 conditionals


In [ ]:
# Define reconnaissance task and targets
task = "Perform subdomain enumeration and basic web discovery"
targets = ["dreadnode.io"]

# Render the operator prompt with targets
prompt = operator.render(task, targets=targets)

print(prompt[:500] + "...")

## 5. Execute Full Agent

Run the complete agent pipeline with error handling and tool integration


In [ ]:
# Configure model (change to your preferred model)
model = "openai/gpt-4.1"  # or "ollama/llama3.2", "anthropic/claude-3-sonnet"
max_steps = 20

# Execute with Strikes tracking
with dn.run("recon-agent", tags=["workshop"]):
    dn.log_params(model=model, max_steps=max_steps, targets=targets)
    dn.log_input("task", task)

    async with bbot_tool:
        # Create the agent pipeline
        chat = await (
            rg.get_generator(model)
            .chat(prompt)
            .catch(*litellm.exceptions.LITELLM_EXCEPTION_TYPES, on_failed="include")
            .watch(rg.watchers.stream_to_logs)
            .using(
                finish_task,  # Task completion tool
                update_todo,  # Todo management
                highlight_for_review,  # Important findings
                bbot_tool,  # BBOT scanning capabilities
                max_depth=max_steps,
            )
            .name("BBOT Reconnaissance Agent")
            .run()
        )

    # Log results
    if chat.failed:
        if isinstance(chat.error, rg.error.MaxDepthError):
            print(f"⚠️  Agent reached max steps ({max_steps})")
            dn.log_metric("max_steps_reached", 1)
        else:
            print(f"❌ Agent failed: {chat.error}")
            dn.log_metric("agent_failed", 1)
    else:
        print("✅ Agent completed successfully")
        dn.log_metric("agent_success", 1)

    dn.log_output("final_conversation", chat.conversation)
    dn.log_metric("total_messages", len(chat.all))
    dn.log_metric("tool_calls", len([m for m in chat.all if m.tool_calls]))